In [3]:
#Load libraries
import os
import torch
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import glob

import gc
from collections import defaultdict
from shutil import copy

In [4]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])


#train_transforms = torchvision.transforms.Compose([
    #torchvision.transforms.Resize((70, 70)),
    #torchvision.transforms.RandomCrop((64, 64)),
    #torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      #])

In [6]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

--2022-08-13 20:25:09--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2022-08-13 20:25:10--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  14.2MB/s    in 5m 47s  

2022-08-13 20:30:57 (13.8 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [7]:
!tar -xvzf food-101.tar.gz

Streaming output truncated to the last 5000 lines.
food-101/images/cannoli/695879.jpg
food-101/images/cannoli/1767088.jpg
food-101/images/cannoli/3201965.jpg
food-101/images/cannoli/3778102.jpg
food-101/images/cannoli/2674065.jpg
food-101/images/cannoli/1169899.jpg
food-101/images/cannoli/1378086.jpg
food-101/images/cannoli/803909.jpg
food-101/images/cannoli/1067240.jpg
food-101/images/cannoli/3002709.jpg
food-101/images/cannoli/2612632.jpg
food-101/images/cannoli/3627612.jpg
food-101/images/cannoli/1798703.jpg
food-101/images/cannoli/560632.jpg
food-101/images/cannoli/512468.jpg
food-101/images/cannoli/2331523.jpg
food-101/images/cannoli/3118302.jpg
food-101/images/cannoli/1499412.jpg
food-101/images/cannoli/1557459.jpg
food-101/images/cannoli/146566.jpg
food-101/images/cannoli/96922.jpg
food-101/images/cannoli/1699562.jpg
food-101/images/cannoli/730051.jpg
food-101/images/cannoli/2177977.jpg
food-101/images/cannoli/2767028.jpg
food-101/images/cannoli/6564.jpg
food-101/images/cannoli/

In [8]:
def prepare_data(filepath, src,dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [9]:
prepare_data('food-101/meta/train.txt', 'food-101/images', 'train')


Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying images into  croque_madam

In [10]:
prepare_data('food-101/meta/test.txt', 'food-101/images', 'test')


Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying images into  croque_madam

In [11]:
#Dataloader

#Path for training and testing directory
train_path= r'train'
test_path= r'test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [12]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [13]:
print(classes)
print(len(classes))

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [14]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [15]:
model=ConvNet(num_classes=101).to(device)

In [16]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [17]:
num_epochs=10
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [18]:
print(train_count,test_count)


75750 25250


In [19]:
from tqdm import tqdm

In [20]:
#Model training and saving best model

best_accuracy=0.0

for epoch in tqdm(range(0,num_epochs)):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    val_loss=0.0
    
    for i, (images,labels) in tqdm(enumerate(train_loader)):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    val_loss=0.0
    test_accuracy=0.0
    for i, (images,labels) in tqdm(enumerate(test_loader)):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        loss1=loss_function(outputs,labels)
        loss1.backward()
        optimizer.step()

        val_loss+= loss1.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    val_loss=val_loss/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Validation Loss: '+str(val_loss)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:07,  7.39s/it]
2it [00:07,  3.37s/it]
3it [00:08,  2.08s/it]
4it [00:09,  1.46s/it]
5it [00:09,  1.13s/it]
6it [00:10,  1.08it/s]
7it [00:10,  1.27it/s]
8it [00:11,  1.41it/s]
9it [00:11,  1.52it/s]
10it [00:12,  1.63it/s]
11it [00:12,  1.68it/s]
12it [00:13,  1.74it/s]
13it [00:13,  1.78it/s]
14it [00:14,  1.79it/s]
15it [00:14,  1.84it/s]
16it [00:15,  1.86it/s]
17it [00:16,  1.71it/s]
18it [00:16,  1.61it/s]
19it [00:17,  1.51it/s]
20it [00:18,  1.48it/s]
21it [00:18,  1.58it/s]
22it [00:19,  1.67it/s]
23it [00:19,  1.74it/s]
24it [00:20,  1.78it/s]
25it [00:20,  1.82it/s]
26it [00:21,  1.82it/s]
27it [00:21,  1.85it/s]
28it [00:22,  1.86it/s]
29it [00:23,  1.86it/s]
30it [00:23,  1.87it/s]
31it [00:24,  1.87it/s]
32it [00:24,  1.87it/s]
33it [00:25,  1.88it/s]
34it [00:25,  1.90it/s]
35it [00:26,  1.91it/s]
36it [00:26,  1.92it/s]
37it [00:27,  1.91it/s]
38it [00:27,  1.91it/s]
39it [00:28,  1.90it/s]
40it [00:28,  1

Epoch: 0 Train Loss: tensor(7.3982) Train Accuracy: 0.00928052805280528 Validation Loss: tensor(10.0029) Test Accuracy: 0.008475247524752476


 10%|█         | 1/10 [12:36<1:53:26, 756.28s/it]
0it [00:00, ?it/s]
1it [00:00,  2.18it/s]
2it [00:00,  2.14it/s]
3it [00:01,  2.20it/s]
4it [00:01,  2.17it/s]
5it [00:02,  2.06it/s]
6it [00:02,  2.11it/s]
7it [00:03,  2.15it/s]
8it [00:03,  2.19it/s]
9it [00:04,  2.22it/s]
10it [00:04,  2.23it/s]
11it [00:05,  2.24it/s]
12it [00:05,  2.24it/s]
13it [00:05,  2.24it/s]
14it [00:06,  2.24it/s]
15it [00:06,  2.23it/s]
16it [00:07,  2.26it/s]
17it [00:07,  2.25it/s]
18it [00:08,  2.28it/s]
19it [00:08,  2.30it/s]
20it [00:08,  2.30it/s]
21it [00:09,  2.29it/s]
22it [00:09,  2.31it/s]
23it [00:10,  2.30it/s]
24it [00:10,  2.27it/s]
25it [00:11,  2.23it/s]
26it [00:11,  2.23it/s]
27it [00:12,  2.24it/s]
28it [00:12,  2.25it/s]
29it [00:12,  2.25it/s]
30it [00:13,  2.24it/s]
31it [00:13,  2.26it/s]
32it [00:14,  2.25it/s]
33it [00:14,  2.24it/s]
34it [00:15,  2.24it/s]
35it [00:15,  2.26it/s]
36it [00:16,  2.26it/s]
37it [00:16,  2.24it/s]
38it [00:16,  2.25it/s]
39it [00:17,  2.21it/s]
40it

Epoch: 1 Train Loss: tensor(5.4803) Train Accuracy: 0.009834983498349836 Validation Loss: tensor(4.9024) Test Accuracy: 0.009227722772277227


 20%|██        | 2/10 [24:59<1:39:50, 748.86s/it]
0it [00:00, ?it/s]
1it [00:00,  2.33it/s]
2it [00:00,  2.31it/s]
3it [00:01,  2.34it/s]
4it [00:01,  2.30it/s]
5it [00:02,  2.31it/s]
6it [00:02,  2.30it/s]
7it [00:03,  2.34it/s]
8it [00:03,  2.36it/s]
9it [00:03,  2.31it/s]
10it [00:04,  2.33it/s]
11it [00:04,  2.32it/s]
12it [00:05,  2.34it/s]
13it [00:05,  2.08it/s]
14it [00:06,  2.16it/s]
15it [00:06,  2.24it/s]
16it [00:07,  2.27it/s]
17it [00:07,  2.28it/s]
18it [00:07,  2.27it/s]
19it [00:08,  2.28it/s]
20it [00:08,  2.30it/s]
21it [00:09,  2.31it/s]
22it [00:09,  2.32it/s]
23it [00:10,  2.30it/s]
24it [00:10,  2.30it/s]
25it [00:10,  2.29it/s]
26it [00:11,  2.32it/s]
27it [00:11,  2.32it/s]
28it [00:12,  2.32it/s]
29it [00:12,  2.29it/s]
30it [00:13,  2.29it/s]
31it [00:13,  2.27it/s]
32it [00:13,  2.28it/s]
33it [00:14,  2.28it/s]
34it [00:14,  2.29it/s]
35it [00:15,  2.31it/s]
36it [00:15,  2.32it/s]
37it [00:16,  2.34it/s]
38it [00:16,  2.34it/s]
39it [00:16,  2.33it/s]
40it

...

Epoch: 7 Train Loss: tensor(4.6389) Train Accuracy: 0.009900990099009901 Validation Loss: tensor(1.5465) Test Accuracy: 0.009663366336633663



0it [00:00, ?it/s]
1it [00:00,  2.24it/s]
2it [00:00,  2.28it/s]
3it [00:01,  2.33it/s]
4it [00:01,  2.34it/s]
5it [00:02,  2.39it/s]
6it [00:02,  2.35it/s]
7it [00:02,  2.36it/s]
8it [00:03,  2.37it/s]
9it [00:03,  2.36it/s]
10it [00:04,  2.37it/s]
11it [00:04,  2.36it/s]
12it [00:05,  2.37it/s]
13it [00:05,  2.34it/s]
14it [00:05,  2.33it/s]
15it [00:06,  2.34it/s]
16it [00:06,  2.30it/s]
17it [00:07,  2.32it/s]
18it [00:07,  2.33it/s]
19it [00:08,  2.32it/s]
20it [00:08,  2.05it/s]
21it [00:09,  1.86it/s]
22it [00:09,  1.86it/s]
23it [00:10,  1.99it/s]
24it [00:10,  2.09it/s]
25it [00:11,  2.17it/s]
26it [00:11,  2.23it/s]
27it [00:12,  2.26it/s]
28it [00:12,  2.28it/s]
29it [00:12,  2.29it/s]
30it [00:13,  2.33it/s]
31it [00:13,  2.32it/s]
32it [00:14,  2.34it/s]
33it [00:14,  2.35it/s]
34it [00:15,  2.35it/s]
35it [00:15,  2.35it/s]
36it [00:15,  2.31it/s]
37it [00:16,  2.31it/s]
38it [00:16,  2.30it/s]
39it [00:17,  2.33it/s]
40it [00:17,  2.34it/s]
41it [00:18,  2.31it/s]
42it 

Epoch: 8 Train Loss: tensor(4.6431) Train Accuracy: 0.009900990099009901 Validation Loss: tensor(24.5130) Test Accuracy: 0.009821782178217822



0it [00:00, ?it/s]
1it [00:00,  2.28it/s]
2it [00:00,  2.33it/s]
3it [00:01,  2.34it/s]
4it [00:01,  2.36it/s]
5it [00:02,  2.35it/s]
6it [00:02,  2.36it/s]
7it [00:02,  2.34it/s]
8it [00:03,  2.33it/s]
9it [00:03,  2.33it/s]
10it [00:04,  2.35it/s]
11it [00:04,  2.36it/s]
12it [00:05,  2.37it/s]
13it [00:05,  2.36it/s]
14it [00:05,  2.34it/s]
15it [00:06,  2.36it/s]
16it [00:06,  2.37it/s]
17it [00:07,  2.37it/s]
18it [00:07,  2.38it/s]
19it [00:08,  2.38it/s]
20it [00:08,  2.38it/s]
21it [00:08,  2.38it/s]
22it [00:09,  2.40it/s]
23it [00:09,  2.40it/s]
24it [00:10,  2.39it/s]
25it [00:10,  2.39it/s]
26it [00:10,  2.39it/s]
27it [00:11,  2.34it/s]
28it [00:11,  2.36it/s]
29it [00:12,  2.37it/s]
30it [00:12,  2.36it/s]
31it [00:13,  2.36it/s]
32it [00:13,  2.38it/s]
33it [00:13,  2.37it/s]
34it [00:14,  2.36it/s]
35it [00:14,  2.36it/s]
36it [00:15,  2.35it/s]
37it [00:15,  2.35it/s]
38it [00:16,  2.33it/s]
39it [00:16,  2.33it/s]
40it [00:16,  2.34it/s]
41it [00:17,  2.34it/s]
42it 

Epoch: 9 Train Loss: tensor(4.6541) Train Accuracy: 0.009900990099009901 Validation Loss: tensor(1.5478) Test Accuracy: 0.009267326732673267
